In [0]:
spark.conf.set(
  "KeyVaultName",
  "Connection String"
)

In [0]:
df_store = spark.read.parquet("abfss://retails-db@test1db.dfs.core.windows.net/raw_data/dbo.stores.parquet")
df_store.show(5)
 

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_products = spark.read.parquet("abfss://retails-db@test1db.dfs.core.windows.net/raw_data/dbo.products.parquet")
df_products.show(5)

+----------+-----------------+-----------+-----+
|product_id|     product_name|   category|price|
+----------+-----------------+-----------+-----+
|         1|   Wireless Mouse|Electronics|  800|
|         2|Bluetooth Speaker|Electronics| 1200|
|         3|         Yoga Mat|    Fitness|  499|
|         4|     Laptop Stand|Accessories|  999|
|         5|     Notebook Set| Stationery|  149|
+----------+-----------------+-----------+-----+
only showing top 5 rows


In [0]:
df_transactions = spark.read.parquet("abfss://retails-db@test1db.dfs.core.windows.net/raw_data/dbo.transactions.parquet")
df_transactions.show(5)

+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_date|
+--------------+-----------+----------+--------+--------+----------------+
|            31|        101|         1|       1|       3|      2025-04-01|
|            32|        102|         2|       2|       2|      2025-04-03|
|            33|        103|         3|       3|       1|      2025-04-05|
|            34|        104|         4|       4|       5|      2025-04-07|
|            35|        105|         5|       5|       2|      2025-04-09|
+--------------+-----------+----------+--------+--------+----------------+
only showing top 5 rows


In [0]:
df_customer = spark.read.parquet("abfss://retails-db@test1db.dfs.core.windows.net/MohammedHameds/Retails_Project/refs/heads/main/dataset/customers.parquet")
df_customer.show(5)

+-----------+----------------+--------------------+------------+-----------------+
|customer_id|       full_name|               email|     country|registration_date|
+-----------+----------------+--------------------+------------+-----------------+
|        101|    Ahmed Khaled|ahmed.khaled1@gma...|       Egypt|       2025-10-16|
|        102| Sara Al Mansour|sara.almansour2@o...|Saudi Arabia|       2025-10-18|
|        103|    Layla Kazemi|layla.kazemi3@yah...|         UAE|       2025-10-19|
|        104|     Omar Farouk|omar.farouk4@gmai...|       Egypt|       2025-10-17|
|        105|Fatima Al Rashid|fatima.alrashid5@...|Saudi Arabia|       2025-10-15|
+-----------+----------------+--------------------+------------+-----------------+
only showing top 5 rows


In [0]:
df_store.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.store_bronze")
df_transactions.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.transactions_bronze")
df_products.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.products_bronze")
df_customer.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.customer_bronze")

In [0]:
from pyspark.sql.functions import col, sum, count

In [0]:
df_customer = df_customer.select(
    col("customer_id").cast("int"),
    col("full_name"),
    col("email"),
    col("country"),
    col("registration_date").cast("date")
)
df_customer.printSchema()
 
 

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)



In [0]:
df_retails_silver = df_transactions.join(df_customer,"customer_id").join(df_products,"product_id").join(df_store, "store_id").withColumn("total_amount", col("quantity") * col("price"))
display(df_retails_silver)
 
 
 
 

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,full_name,email,country,registration_date,product_name,category,price,store_name,location,total_amount
3,8,101,58,3,2025-05-25,Ahmed Khaled,ahmed.khaled1@gmail.com,Egypt,2025-10-16,Desk Organizer,Accessories,399,Tech World Outlet,Qatar,1197
4,9,102,59,5,2025-05-27,Sara Al Mansour,sara.almansour2@outlook.com,Saudi Arabia,2025-10-18,Dumbbell Set,Fitness,1999,Cairo Festival City Mall,Egypt,9995
5,10,103,60,1,2025-05-29,Layla Kazemi,layla.kazemi3@yahoo.com,UAE,2025-10-19,Pen Drive 32GB,Electronics,599,Mega Plaza,Kuwait,599
1,1,104,61,2,2025-06-01,Omar Farouk,omar.farouk4@gmail.com,Egypt,2025-10-17,Wireless Mouse,Electronics,800,City Mall Store,UAE,1600
2,2,105,62,3,2025-06-03,Fatima Al Rashid,fatima.alrashid5@hotmail.com,Saudi Arabia,2025-10-15,Bluetooth Speaker,Electronics,1200,High Street Store,Saudi Arabia,3600
3,3,106,63,4,2025-06-05,Yousef Nasser,yousef.nasser6@gmail.com,UAE,2025-10-16,Yoga Mat,Fitness,499,Tech World Outlet,Qatar,1996
4,4,107,64,5,2025-06-07,Mona Hossam,mona.hossam7@yahoo.com,Egypt,2025-10-18,Laptop Stand,Accessories,999,Cairo Festival City Mall,Egypt,4995
5,5,108,65,1,2025-06-09,Hassan Al Saud,hassan.alsaud8@outlook.com,Saudi Arabia,2025-10-19,Notebook Set,Stationery,149,Mega Plaza,Kuwait,149
1,6,109,66,2,2025-06-11,Ayesha Khan,ayesha.khan9@gmail.com,UAE,2025-10-15,Water Bottle,Fitness,299,City Mall Store,UAE,598
2,7,110,67,3,2025-06-13,Tamer Adel,tamer.adel10@hotmail.com,Egypt,2025-10-17,Smartwatch,Electronics,4999,High Street Store,Saudi Arabia,14997


In [0]:
df_retails_silver.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.retails_silver")

In [0]:
df_country_customer = df_retails_silver.groupBy("country").agg(count("customer_id").alias("Count_Of_Customrs"))
display(df_country_customer)

country,Count_Of_Customrs
Saudi Arabia,17
UAE,16
Egypt,17


In [0]:
df_country_customer.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.country_customer_gold")

In [0]:
df_Contry_transaction = df_retails_silver.groupBy("country", "category").agg(count("transaction_id").alias("Count_transaction"))
display(df_Contry_transaction)

country,category,Count_transaction
Saudi Arabia,Fitness,3
UAE,Stationery,2
UAE,Electronics,5
Egypt,Electronics,7
UAE,Fitness,6
Egypt,Accessories,3
Saudi Arabia,Accessories,4
Egypt,Fitness,6
Egypt,Stationery,1
Saudi Arabia,Electronics,8


In [0]:
df_Contry_transaction.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.Contry_transaction_gold")

In [0]:
import pyspark.sql.functions as F
top_stores_revenue = (

    df_retails_silver
    .groupBy("store_name", "location")
    .agg(F.sum("total_amount").alias("total_revenue"))
    .orderBy(F.desc("total_revenue"))
)
display(top_stores_revenue)



store_name,location,total_revenue
High Street Store,Saudi Arabia,87986
Cairo Festival City Mall,Egypt,55961
City Mall Store,UAE,13786
Tech World Outlet,Qatar,11773
Mega Plaza,Kuwait,8079


In [0]:
top_stores_revenue.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.top_stores_location_revenue_gold")

In [0]:
top_products_sold = df_retails_silver.groupby("product_name").agg(F.sum("quantity").alias("quantity")).orderBy(F.desc("quantity"))
display(top_products_sold)

product_name,quantity
Laptop Stand,22
Desk Organizer,17
Dumbbell Set,17
Bluetooth Speaker,15
Smartwatch,14
Water Bottle,14
Wireless Mouse,12
Pen Drive 32GB,11
Yoga Mat,10
Notebook Set,10


In [0]:
top_products_sold.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.top_products_sold_gold")

In [0]:
top_exp_product = df_retails_silver.groupby("product_name").agg(F.sum("price").alias("price")).orderBy(F.desc("price"))
display(top_exp_product)

product_name,price
Smartwatch,24995
Dumbbell Set,9995
Bluetooth Speaker,6000
Laptop Stand,4995
Wireless Mouse,4000
Pen Drive 32GB,2995
Yoga Mat,2495
Desk Organizer,1995
Water Bottle,1495
Notebook Set,745


In [0]:
top_exp_product.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.top_exp_product_gold")

In [0]:
most_sold_category = df_retails_silver.groupby("category").agg(F.sum("quantity").alias("quantity")).orderBy(F.desc("quantity"))
display(most_sold_category)

category,quantity
Electronics,52
Fitness,41
Accessories,39
Stationery,10


In [0]:
most_sold_category.write.mode("overwrite").saveAsTable("retail_project_alaa.retails.most_sold_category_gold")